<h1 align='center'>Aplicando o aprendizado de máquina à análise de sentimentos</h1>

<p align='center'><img src= https://cibersistemas.pt/wp-content/uploads/2020/09/wall-5.jpeg width='500'></p>


Na era moderna da internet e das mídias sociais, as opiniões, avaliações e recomendações das pessoas se tornaram um recurso valioso para a ciência política e os negócios. Graças às tecnologias modernas, agora podemos coletar e analisar esses dados com mais eficiência. Neste capítulo, vamos nos aprofundar em um subcampo do processamento de linguagem natural (NLP) chamado análise de sentimentos e aprender a usar algoritmos de aprendizado de máquina para classificar documentos com base em sua polaridade: a atitude do escritor. Em particular, vamos trabalhar com um conjunto de dados de 50.000 críticas de filmes do Internet Movie Database (IMDb) e construir um preditor que possa distinguir entre críticas positivas e negativas.

## Preparando os dados de revisão de filme do IMDb para processamento de texto
Como mencionado, a análise de sentimentos, às vezes também chamada de **mineração de opinião**, é uma subdisciplina popular do campo mais amplo da *NLP*; preocupa-se em analisar a polaridade dos documentos. Uma tarefa popular na análise de sentimentos é a classificação de documentos com base nas opiniões ou emoções expressas dos autores em relação a um tópico específico.

In [27]:
import pandas as pd

df = pd.read_csv('movie_data.csv', encoding='utf-8')
df.head(3)

,review,sentiment
0,"Election is a Chinese mob movie, or triads in ...",1
1,I was just watching a Forensic Files marathon ...,0
2,Police Story is a stunning series of set piece...,1


In [28]:
df.shape

(50000, 2)

### Apresentando o modelo de *bag-of-words*

Apresentaremos o modelo *bag-of-words*, que nos permite representar o texto como vetores numéricos de recursos. A ideia por trás do *bag-of-words* é bastante simples e pode ser resumido da seguinte forma:

1. Criamos um vocabulário de tokens exclusivos — por exemplo, palavras — de todo o conjunto de documentos.
2. Construímos um vetor de características de cada documento que contém as contagens de quantas vezes cada palavra ocorre no documento em particular.

Como as palavras únicas em cada documento representam apenas um pequeno subconjunto de todas as palavras no vocabulário do *bag-of-words*, os vetores de recursos consistirão principalmente em zeros, e é por isso que os chamamos de esparsos. Não se preocupe se isso soar muito abstrato; mais pra frente, veremos o passo a passo do processo de criação de um modelo simples de um *bag-of-words*.

### Transformando palavras em vetores de recursos
Para construir um modelo *bag-of-words* baseado na contagem de palavras nos respectivos documentos, podemos usar a classe *CountVectorizer* implementada no *scikit-learn*. Como você verá no código a seguir, o *CountVectorizer* pega uma matriz de dados de texto, que podem ser documentos ou frases, e constrói o modelo *bag-of-words*:

In [29]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer()
docs = np.array([
        'The sun is shining',
        'The weather is sweet',
        'The sun is shining, the weather is sweet, and one and one is two'])
bag = count.fit_transform(docs)

# Agora vamos imprimir o conteúdo do vocabulário para entender melhor os conceitos subjacentes:
print(count.vocabulary_)

{'the': 6, 'sun': 4, 'is': 1, 'shining': 3, 'weather': 8, 'sweet': 5, 'and': 0, 'one': 2, 'two': 7}


Como você pode ver ao executar o comando anterior, o vocabulário é armazenado em um dicionário *Python* que mapeia as palavras exclusivas para índices inteiros. Em seguida, vamos imprimir os vetores de recursos que acabamos de criar:

In [30]:
print(bag.toarray())

[[0 1 0 1 1 0 1 0 0]
 [0 1 0 0 0 1 1 0 1]
 [2 3 2 1 1 1 2 1 1]]


Cada posição de índice nos vetores de recursos mostrados aqui corresponde aos valores inteiros armazenados como itens de dicionário no vocabulário *CountVectorizer*. Por exemplo, a primeira *feature* na posição de índice 0 se assemelha à contagem da palavra 'and', que ocorre apenas no último documento, e a palavra 'is', na posição de índice 1 (a segunda feição nos vetores do documento), ocorre nas três frases. Esses valores nos vetores de características também são chamados de frequências de termos brutos: tf(t, d)— número de vezes que um termo, *t*, ocorre em um documento, **d*. Deve-se notar que, no modelo *bag-of-words*, a ordem das palavras ou dos termos em uma frase ou documento não importa.

A ordem em que as frequências dos termos aparecem no vetor de características é derivada dos índices de vocabulário, que geralmente são atribuídos em ordem alfabética.

<blockquote>
<h4><i>Modelos de N-gram</i></h4>
<p alig='justify'>A sequência de itens no modelo de <i>bag-of-words</i> que acabamos de criar também é chamada de modelo de <i>1-gram</i> ou <i>unigram</i> – cada item ou <i>token</i> no vocabulário representa <u>uma única palavra</u>. De maneira geral, as sequências contíguas de itens no <i>NLP</i>  – palavras, letras ou símbolos – também são chamadas de <i>n-grams</i>. A escolha do número, <i>n</i>, no modelo <i>n-gram</i> depende da aplicação particular; por exemplo, um estudo de <i>Ioannis Kanaris</i> e outros revelou que <i>n-grams</i> de <b>tamanho 3 e 4</b> rendem bons desempenhos na filtragem anti-spam de mensagens de e-mail.</p>

<p alig='justify'>Para resumir o conceito da representação <i>n-gram</i>, as representações de <i>1-gram</i> e <i>2-gram</i> do nosso primeiro documento "the sun is shining" seriam construídas da seguinte forma:
<ul>
<li> <i>1-gram</i>: "the", "sun", "is", "shining"</li>
<li> <i>2-gram</i>: "the sun", "sun is", "is shining"</li>
</ul>
<p alig='justify'>A classe <i>CountVectorizer</i> no <i>scikit-learn</i> nos permite usar diferentes modelos de <i>n-gram</i> por meio de seu parâmetro <i>ngram_range</i>. Embora uma representação de <i>1-gram</i> seja usada por padrão, podemos alternar para uma representação de <i>2-gram</i> inicializando uma nova instância <i>CountVectorizer</i> com <b><i>ngram_range=(2,2)</i></b>.
</blockquote>

### Avaliando a relevância da palavra por meio da frequência do documento inversa da frequência do termo
Quando estamos analisando dados de texto, geralmente encontramos palavras que ocorrem em vários documentos de ambas as classes. Essas palavras que ocorrem com frequência geralmente não contêm informações úteis ou discriminatórias. Agora, falaremos sobre uma técnica útil chamada o **termo de frequência e frequência documento inverso (tf-idf)**, que pode ser usado para reduzir o peso dessas palavras que ocorrem com frequência nos vetores de recursos.

A biblioteca *scikit-learn* implementa ainda outro transformador, a classe `TfidfTransformer`, que pega as frequências de termos brutos da classe *CountVectorizer* como entrada e as transforma em `tf-idfs`:

In [31]:
np.set_printoptions(precision=2)

from sklearn.feature_extraction.text import TfidfTransformer

tfidf = TfidfTransformer(use_idf=True, 
                         norm='l2', 
                         smooth_idf=True)
print(tfidf.fit_transform(count.fit_transform(docs))
      .toarray())

[[0.   0.43 0.   0.56 0.56 0.   0.43 0.   0.  ]
 [0.   0.43 0.   0.   0.   0.56 0.43 0.   0.56]
 [0.5  0.45 0.5  0.19 0.19 0.19 0.3  0.25 0.19]]


Como você viu anteriomente, a palavra 'is' teve a maior frequência de termos no terceiro documento, sendo a palavra mais frequente. No entanto, após transformar o mesmo vetor de características em *tf-idfs*, a palavra 'is' agora está associada a um *tf-idf* relativamente pequeno (0,45) no terceiro documento, pois também está presente no primeiro e no segundo documento e, portanto, é improvável que contenha qualquer informação discriminatória útil. No entanto, se tivéssemos calculado manualmente os *tf-idfs* dos termos individuais em nossos vetores de recursos, teríamos notado que o `TfidfTransformer` calcula o *tf-idfs* de forma ligeiramente diferente em comparação com as equações padrão dos livros didáticos que definimos anteriormente.

### Limpando dados de texto

Aprendemos sobre o modelo *bag-of-words*, frequências de termos e *tf-idfs*. No entanto, o primeiro passo importante antes de construirmos nosso modelo de *bag-of-words* é limpar os dados de texto removendo todos os caracteres indesejados.

In [32]:
# Para ilustrar por que isso é importante, vamos exibir os últimos 500 caracteres
# do primeiro documento no conjunto de dados de revisão de filme reordenado

df.loc[0,'review'][-500:]

"film ends at the 65 or 70-minute mark, there are still a couple big surprises waiting. Simon Yam was my favorite character here and sort of anchors the picture.<br /><br />Election was quite the award winner at last year's Hong Kong Film Awards, winning for best actor (Tony Leung), best picture, best director (Johnny To, who did Heroic Trio!!), and best screenplay. It also had nominations for cinematography, editing, film score (which I loved), and three more acting performances (including Yam)."

Como você pode ver aqui, o texto contém marcação *HTML*, bem como pontuação e outros caracteres que não são letras. Embora a marcação *HTML* não contenha muitas semânticas úteis, os sinais de pontuação podem representar informações úteis e adicionais em determinados contextos de *NLP*. No entanto, para simplificar, agora removeremos todos os sinais de pontuação, exceto os caracteres de *emoticon*, como *:)*, pois certamente são úteis para análise de sentimentos. Para realizar essa tarefa, usaremos a biblioteca *mlxtend*, conforme mostrado aqui:

In [33]:
from mlxtend.text import tokenizer_words_and_emoticons

tokenizer_words_and_emoticons("</a>This :) is :( a test :-)!")

['this', 'is', 'a', 'test', ':)', ':(', ':-)']

<blockquote>
<h5>Lidando com a capitalização da palavra</h5>
<p align="justify">No contexto desta análise, assumimos que a capitalização de uma palavra – por exemplo, se ela aparece no início de uma frase – não contém informações semanticamente relevantes. No entanto, observe que há exceções; por exemplo, removemos a notação de nomes próprios. Mas, novamente, no contexto desta análise, é uma suposição simplificadora que a letra maiúscula não contém informações relevantes para a análise de sentimentos.</p>
</blockquote>

Eventualmente, adicionamos os *emoticons* armazenados temporariamente ao final da *string *do documento processado. Além disso, removemos o caractere de nariz (- em :-)) dos *emoticons* para consistência.

Embora a adição dos caracteres de *emoticon* ao final das sequências de documentos limpados possa não parecer a abordagem mais elegante, devemos observar que a ordem das palavras não importa em nosso modelo de *bag-of-words* se nosso vocabulário consiste em apenas *tokens* de uma palavra.

### Processando documentos em tokens
Depois de preparar com sucesso o conjunto de dados de resenhas de filmes, agora precisamos pensar em como dividir os corpora de texto em elementos individuais. Uma maneira de tokenizar documentos é dividi-los em palavras individuais, dividindo os documentos limpos em seus caracteres de espaço em branco.

No contexto de tokenização, outra técnica útil é retirar o radical das palavras (*word stemming*), que é o processo de transformar uma palavra em sua forma raiz. Ele nos permite mapear palavras relacionadas ao mesmo radical. O algoritmo de *lematização* original foi desenvolvido por *Martin F. Porter*, em 1979, e, portanto, é conhecido como algoritmo de *lematização* de *Porter*. O *Natural Language Toolkit* (NLTK, http://www.nltk.org) para *Python* implementa o algoritmo de determinação de *Porter*, que usaremos no código a seguir:

In [40]:
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()

[porter.stem(word) for word in tokenizer_words_and_emoticons('runners like running and thus they run')]

['runner', 'like', 'run', 'and', 'thu', 'they', 'run']

Usando o *PorterStemmer* do pacote *nltk*, modificamos nossa função *tokenizer* para reduzir as palavras à sua forma raiz, o que foi ilustrado pelo exemplo simples acima em que a palavra *'running'* foi derivada para sua forma raiz *'run'*.

<blockquote>
<h5>Algoritmos de derivação</h5>
<p>O algoritmo de <i>stemming de Porter</i> é provavelmente o algoritmo de <i>stemming</i> mais antigo e simples. Outros algoritmos de lematização populares incluem o mais recente lematizador <i>Snowball</i> (<i>Porter2</i> ou lematizador inglês) e o lematizador <i>Lancaster</i> (lematizador Paice/Husk). Enquanto as raízes <i>Snowball</i> e <i>Lancaster</i> são mais rápidas do que a original <i>Porter</i>, a <i>Lancaster</i> também é notória por ser mais agressiva que a <i>Porter</i>. Esses algoritmos alternativos também estão disponíveis através do pacote NLTK (http://www.nltk.org/api/nltk.stem.html).

<p>Enquanto a <i>Stemming</i> criar palavras não reais, como 'thu' (de 'thus' ("assim", do inglês)), como mostrado no exemplo anterior, uma técnica chamada lematização visa obter as formas canônicas (gramaticalmente corretas) de palavras individuais - as chamadas <i>lemmas</i>. No entanto, a lematização é <b>computacionalmente mais difícil e cara</b> em comparação com a <i>stemming</i> e, na prática, observa-se que a <i>stemming</i> e a lematização tem pouco impacto no desempenho da classificação de texto.</p>
</blockquote>

In [34]:
tokenizer_words_and_emoticons(df.loc[0,'review'][-500:])

['film',
 'ends',
 'at',
 'the',
 '65',
 'or',
 '70',
 'minute',
 'mark',
 'there',
 'are',
 'still',
 'a',
 'couple',
 'big',
 'surprises',
 'waiting',
 'simon',
 'yam',
 'was',
 'my',
 'favorite',
 'character',
 'here',
 'and',
 'sort',
 'of',
 'anchors',
 'the',
 'picture',
 'election',
 'was',
 'quite',
 'the',
 'award',
 'winner',
 'at',
 'last',
 'year',
 's',
 'hong',
 'kong',
 'film',
 'awards',
 'winning',
 'for',
 'best',
 'actor',
 'tony',
 'leung',
 'best',
 'picture',
 'best',
 'director',
 'johnny',
 'to',
 'who',
 'did',
 'heroic',
 'trio',
 'and',
 'best',
 'screenplay',
 'it',
 'also',
 'had',
 'nominations',
 'for',
 'cinematography',
 'editing',
 'film',
 'score',
 'which',
 'i',
 'loved',
 'and',
 'three',
 'more',
 'acting',
 'performances',
 'including',
 'yam']

Por fim, como usaremos os dados de texto limpos repetidamente durante as próximas seções, vamos agora aplicar nossa função de pré-processador a todas as resenhas de filmes em nosso *DataFrame*:

In [35]:
df['review'] = df['review'].apply(tokenizer_words_and_emoticons)
df

,review,sentiment
0,"[election, is, a, chinese, mob, movie, or, tri...",1
1,"[i, was, just, watching, a, forensic, files, m...",0
2,"[police, story, is, a, stunning, series, of, s...",1
3,"[dear, readers, the, final, battle, between, t...",1
4,"[i, have, seen, the, perfect, son, about, thre...",1
...,...,...
49995,"[if, you, see, the, title, 2069, a, sex, odyss...",0
49996,"[there, were, but, two, reasons, for, me, to, ...",0
49997,"[i, saw, this, movie, the, first, seconds, the...",1
49998,"[this, is, another, one, of, those, humans, vs...",0


Antes de passarmos para frente, onde treinaremos um modelo de aprendizado de máquina usando o modelo *bag-of-words*, vamos falar brevemente sobre outro tópico útil chamado **remoção de palavras irrelevantes**. *Stop-words* são simplesmente aquelas palavras que são extremamente comuns em todos os tipos de textos e provavelmente não contêm nenhuma (ou apenas uma pequena) informação útil que possa ser usada para distinguir entre diferentes classes de documentos. Exemplos de *Stop-words* são *is*, *and*, *has* e *like*.

A remoção de *Stop-words* pode ser útil se estivermos trabalhando com frequências de termos brutos ou normalizados em vez de *tf-idfs*, que já estão reduzindo o peso de palavras que ocorrem com frequência. Para remover *Stop-words* das resenhas de filmes, usaremos o conjunto de 127 *Stop-words*, em inglês, que está disponível na biblioteca *NLTK*, que pode ser obtido chamando a função *nltk.download*:

In [45]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\willi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [46]:
from nltk.corpus import stopwords

stop = stopwords.words('english')

[porter.stem(word) for word in tokenizer_words_and_emoticons('a runner likes running and runs a lot')[-10:] if word not in stop]

['runner', 'like', 'run', 'run', 'lot']